In [1]:
import time
from itertools import combinations
import numpy as np
import cvxpy as cp
from opt import *

In [2]:
def matrix_pattern(pattern):
    n = len(pattern)
    pattern = np.reshape(pattern, (1, len(pattern)))
    X = (np.transpose(pattern) @ pattern) / 2
    for i in range(n):
         X[i, i] = pattern[0, i]
    return X

In [3]:
def l1_project(J, h, secure=1, margin=0.0):
    n = len(J)

    K = cp.Variable((n, n), symmetric=True)
    constraints = []
    for i in range(n):
        constraints += [K[i, i] == h[i]]
        for j in range(n):
            if i != j:
                constraints += [K[i, j] <= J[i, j]]
                constraints += [K[i, j] >= 0]

    # add all patterns with single infection
    cascade_pattern = np.ones(n)
    X1 = matrix_pattern(cascade_pattern)

    for k in range(1, secure+1):
        combs = combinations(range(n), k)
        for comb in combs:
            pattern = -1*np.ones(n)
            for el in comb:
                pattern[el] = 1
            X = matrix_pattern(pattern)
            constraints += [cp.sum(cp.multiply(K, X)) >= cp.sum(cp.multiply(K, X1)) + margin]

    # eliminate cascade case
    healthy_pattern = -1*np.ones(n)
    X0 = matrix_pattern(cascade_pattern)
    constraints += [cp.sum(cp.multiply(K, X0)) >= cp.sum(cp.multiply(K, X1)) + margin]

    # add projection objective
    prob = cp.Problem(cp.Minimize(cp.sum(J-K) + cp.sum([h[i]-K[i, i] for i in range(len(J))])), constraints)
    prob.solve(solver=cp.SCS, eps=1e-5)

    new_J = K.value - np.diag(np.diag(K.value))
    new_h = np.diag(K.value)
    print("Optimal value: %s" % prob.value)

    return new_J, new_h

Now we read Seattle data, initialize the Ising model of pandemic, infect node 0 and compute the MAP state. As we can see, all MAP values are +1.

In [4]:
N = np.genfromtxt('seattle_top20_travel_numbers.csv', delimiter=',')
mu = 0.001
h_max = -0.01
J = N * np.log(1 / (1 - mu))
h = h_max * np.ones(len(N))
result0 = gurobi_map_explicit(J, h, [0])
print(result0)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-09-10
Using license file /home/mk/gurobi.lic

Statistics for model map :
  Linear constraint matrix    : 1 Constrs, 20 Vars, 1 NZs
  Variable types              : 0 Continuous, 20 Integer (20 Binary)
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 1.92095, 11.3637 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1, 1 ]
Energy value:  22.44032204866379
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [5]:
# here we compute l1 projection onto the Safe Polytope
start_time = time.time()
projected_J, projected_h = l1_project(J, h, secure=1, margin=0.001)
end_time = time.time()
print('Found projection in {} sec.'.format(end_time-start_time))

Optimal value: 41.69331172435146
Found projection in 1.493504285812378 sec.


/home/mk/.pyenv/versions/3.7.7/envs/opt/lib/python3.7/site-packages/cvxpy/problems/problem.py:1279: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


Now, infecting node 0 does not create any other infections:

In [6]:
result = gurobi_map_explicit(projected_J, projected_h, [0])
print(result)


Statistics for model map :
  Linear constraint matrix    : 1 Constrs, 20 Vars, 1 NZs
  Variable types              : 0 Continuous, 20 Integer (20 Binary)
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 0.398685, 0.398688 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1, 1 ]
Energy value:  1.694667487964156
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0]


However, if two nodes are infected, then again all nodes are infected in MAP state:

In [7]:
result = gurobi_map_explicit(projected_J, projected_h, [0, 1])
print(result)


Statistics for model map :
  Linear constraint matrix    : 2 Constrs, 20 Vars, 2 NZs
  Variable types              : 0 Continuous, 20 Integer (20 Binary)
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 0.398685, 0.398688 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1, 1 ]
Energy value:  1.6936661864880764
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [8]:
# here we compute l1 projection onto the Safe Polytope(2)
start_time = time.time()
projected_J, projected_h = l1_project(J, h, secure=2, margin=0.001)
end_time = time.time()
print('Found projection in {} sec.'.format(end_time-start_time))

Optimal value: 43.62073910350217
Found projection in 3.0805139541625977 sec.


In [9]:
result = gurobi_map_explicit(projected_J, projected_h, [0, 1])
print(result)


Statistics for model map :
  Linear constraint matrix    : 2 Constrs, 20 Vars, 2 NZs
  Variable types              : 0 Continuous, 20 Integer (20 Binary)
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 0.195125, 0.210545 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1, 1 ]
Energy value:  0.7309526900870025
[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0]


In [10]:
# here we compute l1 projection onto the Safe Polytope(3)
start_time = time.time()
projected_J, projected_h = l1_project(J, h, secure=3, margin=0.001)
end_time = time.time()
print('Found projection in {} sec.'.format(end_time-start_time))

Optimal value: 44.308961587876254
Found projection in 11.08910083770752 sec.
